In [17]:
cudaSetDevice(0)

NameError: name 'cudaSetDevice' is not defined

In [ ]:
print(torch.cuda.is_available())
print(torch.version.cuda)

In [1]:
from captum.attr import visualization
import torch
from transformers import BertTokenizer
from Transformer_Explainability.BERT_explainability.modules.BERT.ExplanationGenerator import Generator
from Transformer_Explainability.BERT_explainability.modules.BERT.BertForSequenceClassification import BertForSequenceClassification
from transformers import BertTokenizer
from Transformer_Explainability.BERT_explainability.modules.BERT.ExplanationGenerator import Generator
from transformers import AutoTokenizer


In [2]:
import urllib
import clip
from transformers import BertTokenizer
from Transformer_Explainability.BERT_explainability.modules.BERT.ExplanationGenerator import Generator
from src.data_cleaning.script import *
from src.config.config import *
from src.emotion.go_emotion import *
from src.emotion.model import *
import warnings
warnings.filterwarnings('ignore')
import traceback

In [3]:
subreddit = "for_labeling"

In [4]:
from pathlib import Path  
import pandas as pd
DATA = f'C:/Users/Charlie/Desktop/Database/BERT-Sentiment-Analysis-Reddit-API/DataCleaning/sqldf/{subreddit}.csv'  

In [23]:
emoji = pd.read_csv('C:/Users/Charlie/Desktop/Database/BERT-Sentiment-Analysis-Reddit-API/Emotion-recognition/Emotion-recognition/emoji_mapping_table.csv')
emoji_lst = list(emoji['Emoji'])
try:
    mturk_df = pd.read_csv(DATA,sep=",", encoding='cp1252',on_bad_lines='skip')
except:
    mturk_df = pd.read_csv(DATA,sep=",", encoding='utf-8',on_bad_lines='skip')
mturk_df
image_url_list = []
title_list = []
comments_dict = defaultdict(list)

device = torch.device("cuda:0" if torch.cuda.is_available() else print('hello'))
wt = WordNetTagger()
bert_model = BertForMultiLabelClassification.from_pretrained("monologg/bert-base-cased-goemotions-original").to(device)

bert_model.eval()
tokenizer = BertTokenizer.from_pretrained("monologg/bert-base-cased-goemotions-original")
explanations = Generator(bert_model)

clip_model, preprocess = clip.load("ViT-B/32", device=device, jit=False)


config.num_labels:  28


In [25]:
model = BertForMultiLabelClassification.from_pretrained("monologg/bert-base-cased-goemotions-original").to("cuda")
text_batch = ["This movie was the best movie I have ever seen! some scenes were ridiculous, but acting was great."]
encoding = tokenizer(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].to("cuda")
attention_mask = encoding['attention_mask'].to("cuda")

# true class is positive - 1
true_class = 1

# generate an explanation for the input
expl = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=0)[0]
# normalize scores
expl = (expl - expl.min()) / (expl.max() - expl.min())

# get the model classification
output = torch.nn.functional.softmax(model(input_ids=input_ids, attention_mask=attention_mask)[0], dim=-1)
classification = output.argmax(dim=-1).item()
# get class name
class_name = classifications[classification]
# if the classification is negative, higher explanation scores are more negative
# flip for visualization
if class_name == "NEGATIVE":
  expl *= (-1)

tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
print([(tokens[i], expl[i].item()) for i in range(len(tokens))])
vis_data_records = [visualization.VisualizationDataRecord(
                                expl,
                                output[0][classification],
                                classification,
                                true_class,
                                true_class,
                                1,       
                                tokens,
                                1)]
visualization.visualize_text(vis_data_records)

config.num_labels:  28
[('[CLS]', 0.0), ('This', 0.2158810943365097), ('movie', 0.0899757519364357), ('was', 0.11041862517595291), ('the', 0.35469499230384827), ('best', 0.7750786542892456), ('movie', 0.0709317997097969), ('I', 0.0), ('have', 0.061757270246744156), ('ever', 1.0), ('seen', 0.2806461751461029), ('!', 0.003720361040905118), ('some', 0.20789656043052673), ('scenes', 0.2965109944343567), ('were', 0.23326589167118073), ('ridiculous', 0.6767276525497437), (',', 0.0019471825798973441), ('but', 0.5572431087493896), ('acting', 0.3725723624229431), ('was', 0.07197000086307526), ('great', 0.8070467710494995), ('.', 0.06248350813984871), ('[SEP]', 0.01159625593572855)]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,0 (1.00),1,1.00,"[CLS] This movie was the best movie I have ever seen ! some scenes were ridiculous , but acting was great . [SEP]"


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,0 (1.00),1,1.00,"[CLS] This movie was the best movie I have ever seen ! some scenes were ridiculous , but acting was great . [SEP]"


In [8]:
mturk_df

,SubmissionID,SubmissionTitle,CommentID,Comment,subreddit,Images
0,300001,"Putin, Me, photography",1,I liked the one of Putin himself buried in there,art,https://i.redd.it/7ycc263aruj81.jpg
1,300001,"Putin, Me, photography",2,"Me “Ugh, I hate when people mark posts NSFW fo...",art,https://i.redd.it/7ycc263aruj81.jpg
2,300001,"Putin, Me, photography",3,Love that they all appear to be flacid,art,https://i.redd.it/7ycc263aruj81.jpg
3,300001,"Putin, Me, photography",4,I’m honestly more impressed by the collection....,art,https://i.redd.it/7ycc263aruj81.jpg
4,300001,"Putin, Me, photography",5,Well i can't unzoom in but this was probably t...,art,https://i.redd.it/7ycc263aruj81.jpg
...,...,...,...,...,...,...
2251296,332038,Colossus of the Alps,55,The last one looks like a trip to Auschwitz,wallpaper,https://i.redd.it/k2pt3pao31f31.jpg
2251297,332038,Colossus of the Alps,56,"I remember being about 4 years old, standing u...",wallpaper,https://i.redd.it/k2pt3pao31f31.jpg
2251298,332038,Colossus of the Alps,57,"I'm 60, remember going to get seat belts put i...",wallpaper,https://i.redd.it/k2pt3pao31f31.jpg
2251299,332038,Colossus of the Alps,58,"Just think, folks; the reason why we don’t do ...",wallpaper,https://i.redd.it/k2pt3pao31f31.jpg


In [24]:
save_to_csv = []
start = 300001
count = start-1
end = 300002
for index, m_row in mturk_df.loc[(mturk_df['SubmissionID'] >= start) & (mturk_df['SubmissionID'] <= end)].iterrows():
    print(f'{count}/{end}')
    if m_row['SubmissionID'] != count:
        urllib.request.urlretrieve(m_row['Images'], 'tmp.png')
        image = preprocess(Image.open("tmp.png")).unsqueeze(0).to(device)
        count = m_row['SubmissionID']
    else:
        image = image
        
    sid = m_row['SubmissionID']
    if start <= sid <= end:
        cap_and_comments = [''.join(t for t in m_row['SubmissionTitle'] if str(t) not in emoji_lst)]
        save = mturk_df[["SubmissionID", "SubmissionTitle", "Images"]]
        comment = m_row['Comment']
        
        text = clip.tokenize(cap_and_comments).to(device)
        with torch.no_grad():
            image_features = clip_model.encode_image(image)
            text_features = clip_model.encode_text(text)
            logits_per_image, logits_per_text = clip_model(image, text)
            probs = logits_per_image.softmax(dim=-1).cpu().numpy()
        sorted_idx = [int(i[0]) for i in sorted(enumerate(probs[0]), key=lambda x: x[1], reverse=True)]
        post_labels = []
        
        comment_clean = comment.replace("'", "")

        inputs = tokenizer(comment_clean, return_tensors="pt")
        inputs = inputs.to(device)
        outputs = bert_model(**inputs)
        scores = 1 / (1 + torch.exp(-outputs[0]))  # Sigmoid
        threshold = .5
        comment_labels = []
        for idx, score in enumerate(scores[0]):
            if score > threshold:
                label = bert_model.config.id2label[idx]
                comment_labels.append((label,float(score)))
        if len(comment_labels) == 0:
            idx,score = max(enumerate(scores[0]))
            label = bert_model.config.id2label[idx]
            comment_labels.append((label,float(score)))
        post_labels.append(comment_labels)
    insert_row = [count,comment,post_labels[0]]
    save_to_csv.append(insert_row)

df = pd.DataFrame(save_to_csv)
df.to_csv(f'C:/Users/Charlie/Desktop/Database/BERT-Sentiment-Analysis-Reddit-API/DataCleaning/sqldf/tags2/start_{start}_end_{end}.csv', index=False, encoding='utf8')

300000/300002
300001/300002
300001/300002
300001/300002
300001/300002
300001/300002
300001/300002
300001/300002
300001/300002
300001/300002
300001/300002
300001/300002
300001/300002
300001/300002
300001/300002
300001/300002
300001/300002
300001/300002
300001/300002
300001/300002
300001/300002
300001/300002
300001/300002
300001/300002
300001/300002
300001/300002
300001/300002
300001/300002
300001/300002
300001/300002
300001/300002


KeyboardInterrupt: 

In [12]:
df.to_csv(f'C:/Users/Charlie/Desktop/Database/BERT-Sentiment-Analysis-Reddit-API/DataCleaning/sqldf/tags2/start_{start}_end_{end}.csv', index=False, encoding='utf8')

In [14]:
df

,0,1,2
0,300001,I liked the one of Putin himself buried in there,"[(neutral, 0.9705334901809692)]"
1,300001,"Me “Ugh, I hate when people mark posts NSFW fo...","[(anger, 0.9462706446647644)]"
2,300001,Love that they all appear to be flacid,"[(love, 0.9484241604804993)]"
3,300001,I’m honestly more impressed by the collection....,"[(admiration, 0.9545209407806396), (optimism, ..."
4,300001,Well i can't unzoom in but this was probably t...,"[(neutral, 0.5297912955284119)]"
...,...,...,...
217,300002,The title is the only thing thats really setti...,"[(approval, 0.9055155515670776)]"
218,300002,anyone else reminded of their parents?,"[(neutral, 0.9907659292221069)]"
219,300002,Hardcore agenda pushing in the comments. Much ...,"[(admiration, 0.9908730983734131)]"
220,300002,It should have a half dozen kids peering out t...,"[(neutral, 0.9987926483154297)]"


In [ ]:
result.to_csv(f'C:/Users/Charlie/Desktop/Database/BERT-Sentiment-Analysis-Reddit-API/DataCleaning/sqldf/tags2/{subreddit}_tags.csv', index=False, encoding='utf8')

In [ ]:
import json
import os
from collections import Counter
import matplotlib.pyplot as plt

c = Counter()
with open(f"json_data2/{subreddit}.json", "w") as outfile:
    for file in os.listdir('json_data2'):
        if file.startswith('emo_distribution_start'):
            with open(f'json_data2/{file}') as f:
                data = json.load(f)

                for dct in data:
                    print('\n------------------')
                    print('submission_id: ', dct['submission_id'])
                    print('number_of_comments: ', dct['number_of_comments'])
                    print('relevance_score: ', dct['relevance_score'])
                    c += Counter([j for i in dct['emotion'] for j in i])
                    print('emotion: ', c)
                    print('------------------\n')
    json.dump(c, outfile)

with open(f'json_data2/{subreddit}.json') as f:
    data = json.load(f)
    data = {k: v for k, v in data.items() if k != 'neutral'}
    data = dict(sorted(data.items(), key=lambda item: item[1], reverse=True))

    plt.figure(figsize=(20, 10))

    # Create a bar plot
    plt.barh(list(data.keys()), data.values(), edgecolor='black', linewidth=1.2)

    # Add labels and title
    plt.xticks(size=12)
    plt.yticks(size=12)
    plt.ylabel('Emotion', fontsize=20)
    plt.xlabel('Count', fontsize=20)
    # plt.title('Histogram')
    plt.savefig(f'json_data2/{subreddit}.png')
    # Display the plot
    plt.show()

In [ ]:
lable_table = []
save_to_json = []
start = 0
end = 2
for index, m_row in mturk_df.loc[(mturk_df['index'] >= start) & (mturk_df['index'] <= end)].iterrows():
    try:
        urllib.request.urlretrieve(m_row['image_link'], 'tmp.png')
        image = preprocess(Image.open("tmp.png")).unsqueeze(0).to(device)
        sid = m_row['index']
        if start < sid < end:
            cap_and_comments = [''.join(t for t in m_row['title'] if str(t) not in emoji_lst)]
            save = mturk_df[["index", "title", "image_link"]]
            comments_list = mturk_df.drop(['index', 'title', 'image_link'], axis=1)
            comments = comments_list.iloc[index]
            for entry in comments:
                if str(entry) not in emoji_lst and str(entry) != '0' and str(entry) != 'fill' and str(entry) != '/':
                    comment = ''.join(entry)
                    cap_and_comments.append(comment[:100])
                # print('idx: ', idx, 'comment: ', comment[:200])
            text = clip.tokenize(cap_and_comments).to(device)
            with torch.no_grad():
                image_features = clip_model.encode_image(image)

                text_features = clip_model.encode_text(text)

                logits_per_image, logits_per_text = clip_model(image, text)
                probs = logits_per_image.softmax(dim=-1).cpu().numpy()
            # print(probs)
            sorted_idx = [int(i[0]) for i in sorted(enumerate(probs[0]), key=lambda x: x[1], reverse=True)]
            post_labels = []
            for s in np.array(cap_and_comments)[sorted_idx]:
                inputs = tokenizer(s, return_tensors="pt")
                inputs = inputs.to(device)
                outputs = bert_model(**inputs)
                scores = 1 / (1 + torch.exp(-outputs[0]))  # Sigmoid
                threshold = .5

                comment_labels = []
                for idx, score in enumerate(scores[0]):
                    if score > threshold:
                        label = bert_model.config.id2label[idx]
                        comment_labels.append(label)
                        lable_table.append((save['image_link'][0],s,label))
                post_labels.append(comment_labels)

            print('relevance score: ', probs[0][sorted_idx])
            print('emotion: ', post_labels)

            save_to_json.append(
                {'submission_id': sid,
                 'number_of_comments': len(lable_table),
                 'relevance_score': ' '.join(str(i) for i in probs[0][sorted_idx]),
                 'emotion': post_labels}
            )
            # Write the data to a JSON file
            with open(f'json_data2/emo_distribution_start{start}_end{end}.json', 'w') as f:
                json.dump(save_to_json, f)
            print(f'submission {sid} saved to json!!')

    except Exception as e:
        traceback.print_exc()